# Fantasy Footbal VBD Auction Creator and Optimizer
1. Creates Value-based drafting values from 'Points above replacement' concept
2. Determines optimal split per cost on any position (eg. 80 to RB = 60 RB1; 20 RB2)
3. Runs optimal team splits (% on each position)

Remaining issues:
- QB values are wonky
    - Normalize for prior year values?
    - Manually adjust upwards?
- Optimizating of team splits has too many calculations
    - Narrow search ranges
    - Increase percentage increment?

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [18]:
df_raw_stats = pd.read_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2018 Data\Combined_data.csv')

In [20]:
df_raw_stats = df_raw_stats.fillna(0)
df_raw_stats['Position_Rank'] = df_raw_stats.groupby('Pos')['Exp_PTS'].rank(ascending=False)


In [55]:
df_raw_stats.shape

(720, 19)

### Define league settings

In [218]:
teams = 12
buyin = 200
qb_start, qb_bench = 2,0.25
rb_start, rb_bench = 2.5, .5
wr_start, wr_bench = 2.5, 2
te_start, te_bench = 1, .25
bench_size = 5 ### actual 6, but calculated to optimize for 2 flex. 
d_start = 1
k_start = 1
rb_waiver = int(teams*(rb_start+rb_bench)+1)
wr_waiver = int(teams*(wr_start+wr_bench)+1)
qb_waiver = int(teams*(qb_start+qb_bench)+1)
te_waiver = int(teams*(te_start+te_bench)+1)


## Run assumptions

In [208]:
starters =.90
### position multipliers based on prior years



# Replacement values

In [209]:
# Find replacement-level RB.
#### Replacement = (rb_start+rb_bench)*teams+1
#### Look up points of replacement-level by pos. 
df_rb = df_raw_stats.where(df_raw_stats['Pos']=='RB')
repl_rb=list(df_rb.where(df_rb['Position_Rank']==int(rb_waiver)).dropna().iloc[0])[15]

df_qb = df_raw_stats.where(df_raw_stats['Pos']=='QB')
repl_qb=list(df_qb.where(df_qb['Position_Rank']==int(qb_waiver)).dropna().iloc[0])[15]

df_wr = df_raw_stats.where(df_raw_stats['Pos']=='WR')
repl_wr=list(df_wr.where(df_wr['Position_Rank']==int(wr_waiver)).dropna().iloc[0])[15]

df_te = df_raw_stats.where(df_raw_stats['Pos']=='TE')
repl_te=list(df_te.where(df_te['Position_Rank']==int(te_waiver)).dropna().iloc[0])[15]

In [210]:
df_rb['PAR'] = np.max(df_rb['Exp_PTS']-0,0)
df_rb['PAR'] =df_rb.apply(lambda row: df_rb['Exp_PTS'] -repl_rb,axis=0)
df_rb.loc[df_rb['PAR']<0,'PAR'] = 0

df_wr['PAR'] = np.max(df_wr['Exp_PTS']-0,0)
df_wr['PAR'] =df_wr.apply(lambda row: df_wr['Exp_PTS'] -repl_wr,axis=0)
df_wr.loc[df_wr['PAR']<0,'PAR'] = 0

df_qb['PAR'] = np.max(df_qb['Exp_PTS']-0,0)
df_qb['PAR'] =df_qb.apply(lambda row: df_qb['Exp_PTS'] -repl_qb,axis=0)
df_qb.loc[df_qb['PAR']<0,'PAR'] = 0

df_te['PAR'] = np.max(df_te['Exp_PTS']-0,0)
df_te['PAR'] =df_te.apply(lambda row: df_te['Exp_PTS'] -repl_te,axis=0)
df_te.loc[df_te['PAR']<0,'PAR'] = 0



In [453]:
df_op = df_rb.append(df_qb, ignore_index=True)
df_op = df_op.append(df_te, ignore_index=True)
df_op = df_op.append(df_wr, ignore_index=True)
df_op.dropna()
total_par = np.sum(df_op['PAR'])
value_per_pt = teams * (buyin*starters) / total_par

In [454]:
df_op['Auction Value'] = np.max(df_op['PAR'],0)
df_op['Auction Value'] = df_op.apply(lambda row: df_op['PAR'] * value_per_pt,axis=0)
df_op['Downside'] = (df_op['Lo_PTS']-df_op['Exp_PTS'])*value_per_pt
df_op['Upside'] = (df_op['Hi_PTS']-df_op['Exp_PTS'])*value_per_pt
df_op.dropna()
df_op.sort_values(by=['Auction Value'], ascending=False)
df_pres = df_op[['Pos','Player','Team','Auction Value','Upside','Downside','Exp_PTS','Hi_PTS','Lo_PTS']]
df_pres.sort_values(by=['Auction Value'], ascending=False).head(5)



,Pos,Player,Team,Auction Value,Upside,Downside,Exp_PTS,Hi_PTS,Lo_PTS
0,RB,Todd Gurley,LAR,64.173939,12.761863,-8.459292,276.2,311.2,253.0
1,RB,Le'Veon Bell,PIT,61.585104,5.214133,-6.198619,269.1,283.4,252.1
2,RB,Ezekiel Elliott,DAL,55.605260,9.042691,-7.183106,252.7,277.5,233.0
3,RB,David Johnson,ARI,53.016425,11.886764,-8.495754,245.6,278.2,222.3
4,RB,Alvin Kamara,NO,45.760394,9.042691,-2.698222,225.7,250.5,218.3


In [214]:
df_pres.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2018 Data\Auction_values.csv')

## Optimization function
- Test splits of spending on each position (eg. RB1/RB2) for any given amount. 
- Given these values, then run tests for % contribution to positions


### By position first...

In [499]:
def optimal_split(test_pos,budget):
    if test_pos == "Flex":
        df_temp = df_op.where(np.logical_and(df_op['Pos']!='QB', df_op['Auction Value']>0)).dropna()
    else:
        df_temp = df_op.where(np.logical_and(df_op['Pos']==test_pos, df_op['Auction Value']>0)).dropna()
    points_list = []
    cost_list = []
    
    for pl1 in range(df_temp.shape[0]):
        pl1_name = df_temp.iloc[pl1,1]
        pl1_cost = int(df_temp.iloc[pl1,20])
        pl1_pts = int(df_temp.iloc[pl1,15])
        for pl2 in range(pl1+1,df_temp.shape[0]):
            pl2_name = df_temp.iloc[pl2,1]
            pl2_cost = int(df_temp.iloc[pl2,20])
            pl2_pts = int(df_temp.iloc[pl2,15])
            total_cost = int(pl1_cost + pl2_cost)
            total_pts = int(pl1_pts + pl2_pts)
            cost_list.append(total_cost)
            points_list.append(total_pts)


    df_pairs = pd.DataFrame(np.column_stack([points_list, cost_list]), 
                           columns=['Total Pts', 'Total Cost'])
    df_pairs['Total Pts'] = pd.to_numeric(df_pairs['Total Pts'])
    df_pairs['Total Cost'] = pd.to_numeric(df_pairs['Total Cost'])

    df_max = df_pairs.where(df_pairs['Total Cost']<budget).dropna().reset_index()
    return df_max.iloc[df_max['Total Pts'].idxmax(),:]


In [501]:
def optimal_split_viewer(test_pos,budget):
    if test_pos == "Flex":
        df_temp = df_op.where(np.logical_and(df_op['Pos']!='QB', df_op['Auction Value']>0)).dropna()
    else:
        df_temp = df_op.where(np.logical_and(df_op['Pos']==test_pos, df_op['Auction Value']>0)).dropna()
    points_list = []
    cost_list = []
    pl1_list = []
    pl2_list = []
    pl1_clist = []
    pl2_clist = []
    pl1_plist = []
    pl2_plist = []
    
    for pl1 in range(df_temp.shape[0]):
        pl1_name = df_temp.iloc[pl1,1]
        pl1_cost = int(df_temp.iloc[pl1,20])
        pl1_pts = int(df_temp.iloc[pl1,15])
        for pl2 in range(pl1+1,df_temp.shape[0]):
            pl2_name = df_temp.iloc[pl2,1]
            pl2_cost = int(df_temp.iloc[pl2,20])
            pl2_pts = int(df_temp.iloc[pl2,15])
            total_cost = int(pl1_cost + pl2_cost)
            total_pts = int(pl1_pts + pl2_pts)
            cost_list.append(total_cost)
            points_list.append(total_pts)
            pl1_list.append(pl1_name)
            pl2_list.append(pl2_name)
            pl1_clist.append(pl1_cost)
            pl2_clist.append(pl2_cost)
            pl1_plist.append(pl1_pts)
            pl2_plist.append(pl2_pts)

    df_pairs = pd.DataFrame(np.column_stack([pl1_list, pl2_list, points_list, cost_list, pl1_clist, pl2_clist,pl1_plist,pl2_plist]), 
                           columns=['Player1', 'Player2', 'Total Pts', 'Total Cost','P1 Cost','P2 Cost','P1 Pts','P2 Pts'])
    df_pairs['Total Pts'] = pd.to_numeric(df_pairs['Total Pts'])
    df_pairs['Total Cost'] = pd.to_numeric(df_pairs['Total Cost'])

    df_max = df_pairs.where(df_pairs['Total Cost']<budget).dropna().reset_index()
    return df_max.iloc[df_max['Total Pts'].idxmax(),:]


### Now for whole team... 

In [418]:
# calc min by positions
qb_min = int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='QB', df_op['Auction Value']>0)).dropna()[-2:].sum()+1)
rb_min = max(int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='RB', df_op['Auction Value']>0)).dropna()[-2:].sum()+1),2)
wr_min = max(int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='WR', df_op['Auction Value']>0)).dropna()[-2:].sum()+1),2)
te_min = int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']=='TE', df_op['Auction Value']>0)).dropna()[-2:].sum()+1)
flex_min = max(int(df_op['Auction Value'].where(np.logical_and(df_op['Pos']!='QB', df_op['Auction Value']>0)).dropna()[-2:].sum()+1),2)
print(qb_min,rb_min,wr_min,te_min,flex_min)

9 5 2 1 2


In [434]:
# Create list of position budgets, calculate full state at once. 
# Loop below DOESNT use optimal_split. 
# Optimal split run on df generated by this process! 

qb_max, rb_max, wr_max, te_max, flex_max = 120,140,100,40,60
rem_budget = starters*buyin
team_pts = [] 
team = []
team_cost = []
splits_list=[]
incr = 10

for qx in range(qb_min,min(qb_max,rem_budget),incr):
    rem_budget = starters*buyin
    splits=[]
    for rx in range(rb_min,min(rb_max,int(rem_budget-(qx+wr_min+flex_min+te_min))),incr):
        for wx in range(wr_min,min(wr_max,int(rem_budget-(rx+qx+flex_min+te_min))),incr):
            for fx in range(flex_min,min(flex_max,int(rem_budget-(rx+qx+wx+te_min))),incr):
                for tx in range(te_min,min(te_max,int(rem_budget-(rx+qx+wx+fx))),incr):
                    splits = [qx,rx,wx,fx,tx]
                    if sum(splits)>170:
                        splits_list.append(splits)
                        
                    
#### Drop anything where total < budget to save size. About half is on huge TE budgets... 

In [431]:
run_hours = len(splits_list)/10*2.95/60/60
print(run_hours)

0.7375


In [435]:
len(splits_list)

1956

#### Now make teams! 
- For each list in splits_list, run optimize_split, concat into team
- Teams run list format should be splits plus total points only! 
- Once best are found, can create viewer with additional information

In [445]:
import timeit
start = timeit.default_timer()

# adjuster for expected value. QB seems to go higher by 10-20%.
# In concept, your budget buys you less 1.2x less than expected.
qb_adjust=1.2

splits_list_total_pts = []

for i in range (10):#(len(splits_list)):
    team_pts = optimal_split('QB',splits_list[i][0]/qb_adjust)[3]
    team_pts += optimal_split('RB',splits_list[i][1])[3]
    team_pts += optimal_split('WR',splits_list[i][2])[3]
    team_pts += optimal_split('Flex',splits_list[i][3])[3]
    team_pts +=max(df_op['Exp_PTS'].where(np.logical_and(df_op['Pos']=='TE', df_op['Auction Value']<(splits_list[i][4]))).dropna())
    splits_list_total_pts.append(team_pts)

    
stop = timeit.default_timer()
print((stop-start)/60," minutes")

0.1012502811960682  minutes


In [ ]:
df_results = pd.DataFrame({'QB/RB/WR/Flex/TE':splits_list, 'Team Pts':splits_list_total_pts})
df_results.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2018 Data\Results.csv')

## Team Evaluator
1. Prints players from optimal teams. 

In [503]:
i = 310

qbs = optimal_split_viewer('QB',splits_list[i][0])
rbs = optimal_split_viewer('RB',splits_list[i][1])
wrs = optimal_split_viewer('WR',splits_list[i][2])
flexs = optimal_split_viewer('Flex',splits_list[i][3])
te = df_op.iloc[df_op['Exp_PTS'].where(np.logical_and(df_op['Pos']=='TE', df_op['Auction Value']<(splits_list[i][4]))).dropna().idxmax()]

#df_team = pd.DataFrame(np.column_stack([pl1_list, pl2_list, points_list, cost_list,pl1_costlist,pl2costlist]), 
#                           columns=['Player1', 'Player2', 'Total Pts', 'Total Cost','P1 Cost','P2 Cost'])
names=[qbs[1],qbs[2],rbs[1],rbs[2],wrs[1],wrs[2], te[1], flexs[1],flexs[2]]
costs = [qbs[5],qbs[6],rbs[5],rbs[6],wrs[5],wrs[6], str(int(te[-3])), flexs[5],flexs[6]]
points = [qbs[7],qbs[8],rbs[7],rbs[8],wrs[7],wrs[8], str(int(te[-8])), flexs[7],flexs[8]]

df_team = pd.DataFrame(np.column_stack([names,points, costs]), columns=['Player', 'Points','Costs'])
df_team

,Player,Points,Costs
0,Dak Prescott,252,14
1,Eli Manning,225,4
2,Dalvin Cook,201,36
3,Derrius Guice,152,18
4,Julio Jones,185,34
5,Michael Thomas,167,27
6,Jimmy Graham,105,10
7,Derrius Guice,152,18
8,Mark Ingram,138,13


In [489]:
qbs

index                     59
Player1       Russell Wilson
Player2            Matt Ryan
Total Pts                559
Total Cost                48
P1 Cost                   32
P2 Cost                   16
Name: 19, dtype: object

In [486]:
te

Pos                        TE
Player           Jimmy Graham
Team                       GB
pass_att                    0
pass_comp                   0
pass_yds                    0
pass_td                     0
int                         0
rush                        0
rush_yds                    0
rush_td                     0
fumbles                   0.5
rec                      61.6
rec_yds                 650.3
rec_td                      7
Exp_PTS                 105.9
Hi_PTS                  116.3
Lo_PTS                   93.6
Position_Rank               6
PAR                      29.9
Auction Value         10.9023
Downside             -4.48488
Upside                 3.7921
Name: 1871, dtype: object